<a href="https://colab.research.google.com/github/r73psv/univer/blob/master/ns_avdoshin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
import torchvision
import torchvision.transforms as transforms

In [27]:
#  Загрузка набора MNIST
# преобразовываем изображение в тензоры, затем нормализуем
# путь куда грузим
DATA_PATH = './data'
# выполняемое преобразование над набором данных
trans = transforms.Compose([transforms.ToTensor(),
transforms.Normalize((0.1307,), (0.3081,))])
# грузим набор данных тренировочный
train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True,
transform=trans, download=True)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64,
                                          shuffle=True, num_workers=2)
# грузим набор данных тестовый
test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False,
transform=trans)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                         shuffle=False, num_workers=2)

In [28]:
#задаем устройство на котором будет тенироваться сеть - видеопроцессор, если нет то процессор
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [29]:
#создаем свёрточную НС (4 свёрточных слоя и 2 линейных)
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10,20, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(320, 50, bias=True)
        self.fc2 = nn.Linear(50, 10, bias=True)

    # реализация метода forward в классе Net
    def forward(self, input):
        x = self.pool1(F.sigmoid(self.conv1(input)))
        x = self.pool2(F.sigmoid(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()
net.to(device)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [30]:
input = torch.randn(1, 1, 28, 28)
out = net(input)
print(out.size())
target = torch.LongTensor([3])
loss_fn = nn.L1Loss()
err = loss_fn(out, target)
err.backward()
print(err)
print(net.conv1.weight.data.norm())
print(net.conv1.weight.grad.data.norm())

torch.Size([1, 10])
tensor(2.9971, grad_fn=<MeanBackward0>)
tensor(1.7940)
tensor(0.0038)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [37]:
# задаем параметры НС
input_dim = 784
output_dim = 10
learning_rate = 0.01
criterion = nn.L1Loss()
model = net(784 10)
optimizer = torch.optim.rmsprop(model.parameters(), lr=learning_rate)
seq_dim = 28
iter = 0
for epoch in range(10):
  for i, (images, labels) in enumerate(trainloader):
    model.train()
 # Загрузка изображений в виде тензоров с возможностью накопления градиента
    images = images.view(-1, seq_dim, input_dim).requires_grad_()
 # Обнуление градиента
    optimizer.zero_grad()
 # Прямой проход
    outputs = model(images)
 # Расчитываем функцию потерь
    loss = criterion(outputs, labels)
 # Обратный проход
    loss.backward()
 # Шаг градиентного спуска (изменение параметров НС)
    optimizer.step()
    iter += 1
    if iter % 500 == 0:
 #производим оценки НС
      model.eval()
      correct = 0
      total = 0
      for images, labels in test_loader:
        images = images.view(-1, seq_dim, input_dim)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        accuracy = 100 * correct / total
        print('Итерация: {}. Потери: {}. Точность: {}'.format(iter, loss.item(), accuracy))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-37-55ff1bdc8d7e>, line 6)